Para montar vuestra unidad drive siempre que lo necesitéis:


In [0]:
# montamos la unidad drive donde tenemos los datos en la carpeta drive/My Drive
from google.colab import drive
drive.mount('/content/drive')

# Preparamos los datos

In [0]:
# por compatibilidad, necesitamos TF 1.x
%tensorflow_version 1.x

In [0]:
# Descargad solo las imágenes del dataset CelebA (https://drive.google.com/open?id=0B7EVK8r0v71pZjFTYXZWM3FlRnM) y subidlo a FileBin.com
# Luego descargadlo y descomprimidlo
# !wget URL
# unzip img_align_celeba.zip

In [0]:
# también podéis hacerlo montando vuestra unidad tras haber guardado una copia del arhivo en ella
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# al haber montado la unidad en /content/drive, los archivos deberían estar disponibles ahí.
# también podéis utilizar el menú de la izquierda, haciendo click en la carpetita, para
# explorar vuestra estructura de archivos
!ls drive/My\ Drive

In [0]:
# descomprimimos las imágenes
!unzip drive/My\ Drive/img_align_celeba.zip

In [0]:
# comprobamos que las tenemos disponibles en la carpeta actual
!ls -lah

In [0]:
# vamos a cargar y a mostrar algunas de las imágenes
import os
import numpy as np
from PIL import Image
# configuramos Colab para que nos muestre las imágenes más grandes
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)

In [0]:
# cargamos las imágenes como RGB y las convertimos a array
def load_image(filename):
	image = Image.open(filename)
	image = image.convert('RGB')
	pixels = np.asarray(image)
	return pixels

# cargamos las imágenes dentro de un directorio
def load_faces(directory, n_faces):
	faces = list()
	# listamos los archivos
	for filename in os.listdir(directory):
		# cargamos la imagen
		pixels = load_image(directory + filename)
		# la guardamos
		faces.append(pixels)
		# paramos cuando llegamos a n_faces
		if len(faces) >= n_faces:
			break
	return np.asarray(faces)

# para visualizar las imagenes
def plot_faces(faces, n):
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(faces[i])
	plt.show()

# carpeta donde estan nuestras imagenes descomprimidas
directory = 'img_align_celeba/'
# cargamos 25 imagenes para visualizarlas
faces = load_faces(directory, 25)
print('Loaded: ', faces.shape)
# y las mostramos
plot_faces(faces, 5)

Ahora tenemos que extraer únicamente la cara de esas imágenes y guardarlas como nuestro nuevo dataset.

In [0]:
# vamos a utilizar MTCNN, que es un detector de caras, para extraer solo la
# porción de la imagen que contenga una cara, y luego guardaremos el dataset
!pip install mtcnn
from mtcnn.mtcnn import MTCNN

# extraemos la cara de una imagen y la guardamos
def extract_face(model, pixels, required_size=(80, 80)):
	# detectamos la cara
	faces = model.detect_faces(pixels)
	# si no hay cara, paramos
	if len(faces) == 0:
		return None
	# obtenemos las dimensiones de la cara detectada
	x1, y1, width, height = faces[0]['box']
	# esto es un "apaño" para un bug que tiene la librería (a veces devuelve coords < 0)
	x1, y1 = abs(x1), abs(y1)
	# convertimos a coordenadas
	x2, y2 = x1 + width, y1 + height
	# cortamos la cara
	face_pixels = pixels[y1:y2, x1:x2]
	# redimensionamos al tamaño deseado
	image = Image.fromarray(face_pixels)
	image = image.resize(required_size)
	face_array = np.asarray(image)
	return face_array

# cargamos imágenes y extraemos la cara con la función anterior
def load_faces(directory, n_faces):
	# preparamos el modelo que detecta las caras
	model = MTCNN()
	faces = list()
	# recorremos todos los archivos disponibles
	for filename in os.listdir(directory):
		# cargamos la imagen
		pixels = load_image(directory + filename)
		# detectamos la cara
		face = extract_face(model, pixels)
		if face is None:
			continue
		# la guardamos
		faces.append(face)
		print(len(faces), face.shape)
		# paramos cuando ya tenemos n_faces
		if len(faces) >= n_faces:
			break
	return np.asarray(faces)

# carpeta con todas las imágenes
directory = 'img_align_celeba/'
# carga y extrae las caras de N imagenes
all_faces = load_faces(directory, 50000)
print('Loaded: ', all_faces.shape)

In [0]:
# guardamos el dataset para que sea más cómodo en el futuro
np.savez_compressed('img_align_celeba.npz', all_faces)

In [0]:
# lo copiamos a nuestro Drive para no perderlo
!cp img_align_celeba.npz drive/My\ Drive/img_align_celeba.npz
!ls -lah drive/My\ Drive/

In [0]:
# si queremos recuperar el dataset, es así de fácil
import numpy as np
data = np.load('drive/My Drive/img_align_celeba.npz')
faces = data['arr_0']
print('Loaded: ', faces.shape)

¡Ya tenemos el dataset listo! Vamos a implementar nuestra GAN. Os daréis cuenta de que es siempre lo mismo, con ligeras variaciones dependiendo del tamaño de las imágenes:

# Creamos nuestra GAN y la entrenamos

In [0]:
# importamos las librerías necesarias
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)

In [0]:
# definimos el discriminador
def define_discriminator(in_shape=(80,80,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(128, (5,5), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample (por el atributo 'strides')
	model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
 	# downsample
	model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# clasificador
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compilamos modelo
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [0]:
# definimos el generador
def define_generator(latent_dim):
	model = Sequential()
	n_nodes = 128 * 5 * 5
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((5, 5, 128)))
	# upsample a 10x10
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample a 20x20
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample a 40x40
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
 	# upsample a 80x80
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# salida (nuestra imagen fake)
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [0]:
# definimos el modelo GAN combinando generador y discriminador, para entrenar el generador
def define_gan(g_model, d_model):
    # Así que congelamos el discriminador:
    d_model.trainable = False
    # ahora conectamos el G(z) al D(x)
    model = Sequential()
    # añadimos el generador primero: él es el encargado de generar una muestra
    # a partir del espacio latente
    model.add(g_model)
    # y el discriminador después: le introducimos la muestra generada por el 
    # G(z) para que nos diga si cree que es real o fake
    model.add(d_model)
    # y ahora sí, compilamos el modelo
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5)) 
    return model

In [0]:
# definimos las funciones para cargar los datos
def load_real_samples():
    # cargamos los datos
    print('Loading data... this may take a while!')
    data = np.load('drive/My Drive/img_align_celeba.npz')
    print('Loading finished!')
    X = data['arr_0']
    # convertimos a float32
    X = X.astype('float32')
    # escalamos entre -1 y 1
    X = (X - 127.5) / 127.5
    return X

# nos creamos una función que nos devuelva n_samples del dataset con sus etiquetas (1) 
def generate_real_samples(dataset, n_samples):
    # seleccionamos n_samples muestras aleatoriamente
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # las cogemos
    X = dataset[ix]
    # generamos las etiquetas reales (1)
    y = np.ones((n_samples, 1))
    return X, y

In [0]:
# generamos los vectores latentes que introduciremos al generador
def generate_latent_points(latent_dim, batch_size):
    # generamos un vector de batch_size * latent_dim números aleatorios
    # latent_dim es la dimensión del vector latente
    # batch_size es el número de elementos por batch
    x_input = np.random.randn(latent_dim * batch_size)
    # redimensionamos el vector para que tenga un tamaño (batch_size, latent_dim)
    x_input = x_input.reshape(batch_size, latent_dim)
    return x_input

# creamos datos fake con el generador (dinero falsificado)
def generate_fake_samples(g_model, latent_dim, n_samples): 
    # usamos la función anterior para generar los vectores latentes que 
    # necesitamos para generar muestras fake
    x_input = generate_latent_points(latent_dim, n_samples)
    # le introducimos los vectores latentes al generador para obtener
    # muestras similares a las reales
    X = g_model.predict(x_input)
    # le asignamos la etiqueta 1 (porque utilizaremos esta función para
    # entrenar el G y en ese caso queremos "engañar" al D)
    y = np.zeros((n_samples, 1)) 
    return X, y

In [0]:
# función para guardar las imágenes generadas
def save_plot(examples, epoch, n=7):
	# escalamos de [-1,1] (la salida de nuestra gan, debido a la función de activación tanh) a [0,1]
	examples = (examples + 1) / 2.0
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i])
	# guardamos las imágenes
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	plt.savefig(filename)
	plt.close()

In [0]:
# función para entrenar la GAN: el discriminador y el generador
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # bucle para las epochs
    for epoch in range(n_epochs):
        # bucle para los batch
        for batch in range(bat_per_epo):
            
            # en esta ocasión vamos a separar las pérdidas del discriminador
            # cuando le metemos imágenes reales y cuando le metemos imágenes
            # fake para ver cómo lo hace con cada tipo
            # recordad que lo ideal es que llegue a un 50% de acc en cada uno

            # preparamos los datos reales
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # actualizamos el discriminador
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            
            # generamos datos falsos
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # actualizamos el discriminador
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			
            # preparamos los puntos en el espacio latente: serán la entrada al
            # modelo GAN con el que entrenaremos el generador
            X_gan = generate_latent_points(latent_dim, n_batch)
            
            # creamos etiquetas invertidas para el generador: utilizamos el D(x) 
            # para que piense que las muestras que le introducimos son reales, y
            # en caso de que diga que no son reales, aprovechamos la información
            # de sus gradientes para actualizar el G(z) para que la próxima vez
            # los datos generados por G(z) sean más plausibles (parecidos a los 
            # reales)
            y_gan = np.ones((n_batch, 1))
            
            # como acabamos de ver, entrenamos el generador de forma que actualice
            # sus pesos usando los gradientes del discriminador
            # tened en cuenta que en este modelo (gan_model) el discriminador está
            # congelado, por lo que no se actualizan sus pesos: no queremos "untar"
            # a nuestro policía, lo que queremos es fabricar dinero más realista.
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            # mostramos el progreso
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % (epoch+1, batch+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (epoch+1) % 10 == 0 or epoch == 0:
            # preparamos ejemplos reales
            X_real, y_real = generate_real_samples(dataset, n_batch)
            # evaluamos el discriminador con datos reales
            _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
            # preparamos ejemplos fake
            x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
            # evaluamos el discriminador con datos fake
            _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
            # mostramos cómo de bueno es nuestro policía
            print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
            # guardamos las imágenes generadas
            save_plot(x_fake, epoch)
            # guardamos el generador para tenerlo disponible más tarde
            filename = 'generator_model_%03d.h5' % (epoch + 1)
            g_model.save(filename)

In [0]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=150, n_batch=256)

# Guardamos los modelos e imagenes en nuestro Drive

In [0]:
# veamos los archivos generados
!ls -lah

In [0]:
# los guardamos en nuestro drive para evitar tener que reejecutar cada vez
!mkdir drive/My\ Drive/6_latent_space
!cp *gen* drive/My\ Drive/6_latent_space/
!ls -lah drive/My\ Drive/6_latent_space/

# Visualizamos los datos que es capaz de generar nuestra GAN

In [0]:
import numpy as np
latent_dim = 100
n_samples = 100

# definimos el mismo código latente para todos los tests
latent_points = generate_latent_points(latent_dim, n_samples)
# y los guardamos
np.savez_compressed('latent_points.npz', latent_points)

In [0]:
g_model = define_generator(latent_dim)
g_model.load_weights('generator_model_010.h5')
X = g_model.predict(latent_points)

In [0]:
X.shape

In [0]:
def plot_results(imgs, n_rows, n_cols, epoch, h=80, w=80):
    imgs = (imgs + 1) / 2.0
    output_img = np.zeros((n_rows*h, n_cols*w, 3), np.float)
    k = 0
    for i in range(n_rows):
        for j in range(n_cols):
            output_img[h*i:h*(i+1), w*j:w*(j+1), :] = imgs[k]
            k += 1
    plt.imshow(np.asarray(output_img*255., np.uint8))

In [0]:
plot_results(X, 8, 8, 10)

In [0]:
# veamos el modelo 50
g_model = define_generator(latent_dim)
g_model.load_weights('generator_model_050.h5')
X = g_model.predict(latent_points)
plot_results(X, 8, 8, 100)

In [0]:
# el 100
g_model = define_generator(latent_dim)
g_model.load_weights('generator_model_100.h5')
X = g_model.predict(latent_points)
plot_results(X, 8, 8, 100)

In [0]:
# el 150
g_model = define_generator(latent_dim)
g_model.load_weights('generator_model_150.h5')
X = g_model.predict(latent_points)
plot_results(X, 8, 8, 100)

# Interpolando entre dos puntos de nuestro espacio latente...

In [0]:
import tensorflow as tf

# uniform interpolation between two points in latent space
def interpolate_points(p1, p2, n_steps=10):
	# interpolate ratios between the points
	ratios = np.linspace(0, 1, num=n_steps)
	# linear interpolate vectors
	vectors = list()
	for ratio in ratios:
		v = (1.0 - ratio) * p1 + ratio * p2
		vectors.append(v)
	return np.asarray(vectors)

# create a plot of generated images
def plot_generated(examples, n, figsize=(20, 20)):
	plt.figure(figsize=figsize)
    # plot images
	for i in range(n):
		# define subplot
		plt.subplot(1, n, 1 + i)
		# turn off axis
		plt.axis('off')
		# plot raw pixel data
		plt.imshow(examples[i, :, :])
	plt.show()

# cargamos el modelo
model = tf.keras.models.load_model('generator_model_050.h5')
# obtenemos la codificación de 2 puntos en nuestro espacio latente
pts = generate_latent_points(100, 2)
# los interpolamos
interpolated = interpolate_points(pts[0], pts[1])
# generamos las imágenes
X = model.predict(interpolated)
# escalamos de [-1,1] a [0,1] (por la salida tanh)
X = (X + 1) / 2.0
# mostramos los resultados
plot_generated(X, len(interpolated), figsize=(24, 24))

# Ejemplo de "vector arithmetic"

In [0]:
# create a plot of generated images
def plot_generated(examples, rows, cols, figsize=(20,20)):
	# plot images
	plt.figure(figsize=figsize)
	for i in range(rows * cols):
		# define subplot
		plt.subplot(rows, cols, 1 + i)
		# turn off axis
		plt.axis('off')
		# plot raw pixel data
		plt.imshow(examples[i, :, :])
	plt.show()

# load model
model = tf.keras.models.load_model('generator_model_150.h5')

# load the saved latent points
data = np.load('latent_points.npz')
points = data['arr_0']

# creamos las imágenes con esos vectores latentes
images = model.predict(points)

# representamos
images = (images + 1) / 2.0
plot_generated(images, 10, 10)

In [0]:
# elegimos 3 representantes de:
# mujeres sonriendo
# mujeres neutrales
# hombres neutrales

# average list of latent space vectors
def average_points(points, ix):
	# convert to zero offset points
	zero_ix = [i-1 for i in ix]
	# retrieve required points
	vectors = points[zero_ix]
	# average the vectors
	avg_vector = np.mean(vectors, axis=0)
	# combine original and avg vectors
	all_vectors = np.vstack((vectors, avg_vector))
	return all_vectors

# los representantes
smiling_woman_ix = [14, 20, 45]
neutral_woman_ix = [8, 24, 77]
neutral_man_ix = [5, 42, 93]

# load the saved latent points
data = np.load('latent_points.npz')
points = data['arr_0']

# average vectors
smiling_woman = average_points(points, smiling_woman_ix)
neutral_woman = average_points(points, neutral_woman_ix)
neutral_man = average_points(points, neutral_man_ix)

# combine all vectors
all_vectors = np.vstack((smiling_woman, neutral_woman, neutral_man))

# generate images
images = model.predict(all_vectors)

# scale pixel values
images = (images + 1) / 2.0
plot_generated(images, 3, 4)

# smiling woman - neutral woman + neutral man = smiling man
result_vector = smiling_woman[-1] - neutral_woman[-1] + neutral_man[-1]

# generate image
result_vector = np.expand_dims(result_vector, 0)
result_image = model.predict(result_vector)

# scale pixel values
result_image = (result_image + 1) / 2.0
plt.figure(figsize=(5, 5))
plt.imshow(result_image[0])
plt.title('Man smiling')
plt.axis('off')
plt.show()